In [1]:
store_path = "data"
SAVER = True #False
#root = "/home/sebasmos/Desktop/DATASETS/pix2pix/val"
#root = "/n/pfister_lab2/Lab/vcg_biology/ORION/ORION-PATCH/C1-C40-patches/CRC06"
#root = "/net/coxfs01/srv/export/coxfs01/pfister_lab2/share_root/Lab/scajas/DATASETS/DATASET_pix2pix_merged/DATAFULL/train"


In [2]:
# CRC01
!ls /net/coxfs01/srv/export/coxfs01/pfister_lab2/share_root/Lab/scajas/DATASETS/DATASET_pix2pix/train
!ls /n/pfister_lab2/Lab/vcg_biology/ORION/ORION-IF-1

full_img = "/n/pfister_lab2/Lab/vcg_biology/ORION/ORION-IF-1/P37_S34_A24_C59kX_E15"
root = "/net/coxfs01/srv/export/coxfs01/pfister_lab2/share_root/Lab/scajas/DATASETS/DATASET_pix2pix/train"
save_dir = "/net/coxfs01/srv/export/coxfs01/pfister_lab2/scajas/pytorch-CycleGAN-and-pix2pix_master/CRC01"

img_he	img_if	mask
Celltable_C1.csv
Celltable_C1-YC.csv
Celltable_C2.csv
Celltable_C3.csv
Celltable_C40B1.csv
Celltable_C40B2.csv
Celltable_C4.csv
Celltype_OrionCRC_20220211.xlsx
P37_S24_Full_A24_C59kX_E15@20220104_223653_988272.ome.tiff
P37_S25_Full_A24_C59kX_E15@20220105_001442_622814.ome.tiff
P37_S25_Scan_20220114_131907_01x3x00400.rcpnl
P37_S26_Lung_A24_C59kX_E15@20220105_224440_730242.ome.tiff
P37_S26_NSCLC_A24_C59kX_E15@20220105_001550_901031.ome.tiff
P37_S27_Full_A24_C59kX_E15@20220106_002943_210932.ome.tiff
P37_S28_A24_C59kX_E15@20220106_014116_878434.ome.tiff
P37_S29_A24_C59kX_E15@20220106_014304_946511.ome.tiff
P37_S30_A24_C59kX_E15@20220106_014319_409148.ome.tiff
P37_S31_A24_C59kX_E15@20220106_014409_014236.ome.tiff
P37_S32_A24_C59kX_E15@20220106_014630_553652.ome.tiff
P37_S33_A24_C59kX_E15@20220107_180446_881530.ome.tiff
P37_S34_A24_C59kX_E15@20220107_202112_212579.ome.tiff
P37_S35_A24_C59kX_E15@20220108_012037_490594.ome.tiff
P37_S36_A24_C59kX_E15@20220108_012058_082564.om

In [3]:
import os
os.listdir(root)
import copy
import os
import torchvision
import torchvision.transforms as T
import skimage.exposure
import torch
from PIL import Image

import torch
import torch.utils.data
import torchvision


#import transforms as T
import torchvision.transforms as visionT
import pdb
import numpy as np
import cv2

from skimage import io

import glob

import random
import tifffile
import pickle
import time
import matplotlib.pyplot as plt


if not os.path.exists(store_path):
    os.makedirs(store_path)
else:
    print('folder already exists')

def plot_imgs(imgs, titles):
    """
    Generate visualization of list of arrays
    :param imgs: list of arrays, each numpy array is an image of size (width, height)
    :param titles: list of titles [string]
    """
    # create figure
    fig = plt.figure(figsize=(50, 50))
    # loop over images
    for i in range(len(imgs)):
        fig.add_subplot(4, 4, i + 1)
        plt.imshow(imgs[i])
        plt.title(str(titles[i]))
        plt.axis("off")

import torchvision
import torchvision.transforms as T
#import transforms as T # Custom version 
import PIL
import skimage.exposure


folder already exists


### Calculating minimum and Maximum based on UC folders

In [ ]:

class Dataloader_vcg_sizer(torch.utils.data.Dataset): 
    def __init__(self, root, 
                 Check_files = False, 
                 mask_flag=False, 
                 augment=False, 
                 transforms_he=None, 
                 transforms_if=None):
        self.root = root
        self.Check_files = Check_files
        self.augment = augment
        self.transforms_he = transforms_he
        self.transforms_if = transforms_if
        self.mask_flag = mask_flag
        self.imgs = list(sorted([logo_name for i, logo_name in enumerate(os.listdir(os.path.join(root, "img_he"))) if ".tif" in logo_name]  
))# HE
        self.targets = list(sorted([logo_name for i, logo_name in enumerate(os.listdir(os.path.join(root, "img_if"))) if ".tif" in logo_name]  
))# IF  
        #self.masks = list(sorted([logo_name for i, logo_name in enumerate(os.listdir(os.path.join(root, "mask"))) if ".tif" in 
    def __getitem__(self, idx):
        ## Data paths 
        # https://syspharm.slack.com/archives/C02SC9VS7AA/p1663364492474239 
        img_path = os.path.join(self.root, "img_he", self.imgs[idx])        
        targets_path = os.path.join(self.root, "img_if", self.targets[idx])
        #mask_path = os.path.join(self.root, "mask", self.targets[idx])
        #print(targets_path)
        # Read images
        img = tifffile.imread(img_path)
        target = tifffile.imread(targets_path)#.astype("float32")
        target = skimage.util.img_as_float32(target)
        # Normalize images
        #CHANNELS = (0, 3, 17)
        img = np.moveaxis(img, 0, 2)
        channels = range(19)#[0,3,17] # visual markers
        target = np.array([target[ch,:,:] for ch in channels]).astype("float32")
        target = target.transpose(1,2,0)
        #target=torch.as_tensor(target.astype("float32"))

        #print(target.shape)
        """
        target = np.dstack([
            skimage.exposure.rescale_intensity(
                target[c],
                in_range=(np.percentile(target[c], 1), np.percentile(target[c], 99.9)),
                out_range=(0, 1)
            ) 
            for c in CHANNELS
        ]).astype(np.float32)
        """
        if self.augment is not None:        
            img, target = self.transforms_he(img), self.transforms_if(target)
            return img, target
        
    def __len__(self):
        return len(self.imgs)
    
def get_transform(train, size=256, HE_IF = "he"):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        if HE_IF=="he":
            transforms.append(T.Resize((size,size)))
        elif HE_IF=="if":
            transforms.append(T.Resize((size,size)))
        else:
            transforms.append(T.Resize((size,size)))
        
    return T.Compose(transforms)

transforms_he =  get_transform(train= True, size=256, HE_IF = "he")
transforms_if = get_transform(train=True, size=256 , HE_IF = "if")

dataset_test = Dataloader_vcg_sizer(root, 
                                    Check_files = False, 
                                    mask_flag= False, 
                                    augment=True, 
                                    transforms_he=transforms_he, 
                                    transforms_if=transforms_if)

print("Dataset size: ", len(dataset_test), "HE-IF pairs")

dataloader = dataset_test 
num_batches = 0

dict_HE = {"min": np.ones((3,1)), "max":np.zeros((3,1))}


def get_minmax(x):
    n = x.shape[0]
    minimo = np.ones((n,1))
    maximo = np.zeros((n,1))
    for ch in range(x.shape[0]):
            if np.array(x[ch,:,:].min())<minimo[ch,:]:
                minimo[ch,:] = x[ch,:,:].min()
            if np.array(x[ch,:,:].max())>maximo[ch,:]:
                maximo[ch,:] = x[ch,:,:].max()
    return minimo, maximo

def calculate_minmax(dataloader):
    _he_min=0
    _he_max=0
    _if_min=0
    _if_max=0
    batches = 0
    for idx in range(len(dataloader)):
        img, target = dataset_test[idx]
        #print(f"Range for HE]-> [{img.min(), img.max()}] - Shape: [{img.shape}]")
        #print(f"Range for IF]-> [{target.min(), target.max()}] - Shape: [{target.shape}]")
        minimoHE, maximoHE = get_minmax(img)
        minimoIF, maximoIF = get_minmax(target)#target.min(axis=(1,2)), target.max(axis=(1,2))#
        _he_min +=minimoHE
        _he_max +=maximoHE
        _if_min +=minimoIF
        _if_max +=maximoIF
        batches+=1
    return _if_min / batches, _if_max / batches 
#import json
#out_file = open("minmax.json","w")

min,max = calculate_minmax(dataloader)
print("Method 1".center(60,"-"))
print("IF MAX: ",max.T)
print("IF MIN: ", min.T )
#print(json.dumps(f))
#json.dump(f, "minmax.json", ident=6)
#out_file.close()
maximoIF = np.mean([np.array(dataset_test[idx][1]).max(axis=(1,2)) for idx in range(len(dataloader))], axis=0)
minIF = np.mean([np.array(dataset_test[idx][1]).min(axis=(1,2)) for idx in range(len(dataloader))], axis=0)
print("Method 2".center(60,"-"))
print("IF MAX: ",maximoIF)
print("IF MIN: ", minIF )

### Calculating minimum based on Percentile and the average image.

In [ ]:
class Dataloader_vcg_sizer(torch.utils.data.Dataset): 
    def __init__(self, root, 
                 Check_files = False, 
                 mask_flag=False, 
                 augment=False, 
                 transforms_he=None, 
                 transforms_if=None):
        self.root = root
        self.Check_files = Check_files
        self.augment = augment
        self.transforms_he = transforms_he
        self.transforms_if = transforms_if
        self.mask_flag = mask_flag
        self.imgs = list(sorted([logo_name for i, logo_name in enumerate(os.listdir(os.path.join(root, "img_he"))) if ".tif" in logo_name]  
))# HE
        self.targets = list(sorted([logo_name for i, logo_name in enumerate(os.listdir(os.path.join(root, "img_if"))) if ".tif" in logo_name]  
))# IF  
    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "img_he", self.imgs[idx])        
        targets_path = os.path.join(self.root, "img_if", self.targets[idx])
        img = tifffile.imread(img_path)
        target = tifffile.imread(targets_path)
        #target = skimage.util.img_as_float32(target)
        #CHANNELS = (0, 3, 17)
        
        img = np.moveaxis(img, 0, 2)
        channels = range(19)#[0,3,17] # visual markers
        target = np.array([target[ch,:,:] for ch in channels])#.astype("float32")
        target = target.transpose(1,2,0)
        #target=torch.as_tensor(target.astype("float32"))

        #print(target.shape)
        """
        target = np.dstack([
            skimage.exposure.rescale_intensity(
                target[c],
                in_range=(np.percentile(target[c], 1), np.percentile(target[c], 99.9)),
                out_range=(0, 1)
            ) 
            for c in CHANNELS
        ]).astype(np.float32)
        """
        return img, target
        
    def __len__(self):
        return len(self.imgs)
    
dataset_train = Dataloader_vcg_sizer(root, 
                                    Check_files = False, 
                                    mask_flag= False, 
                                    augment=True, 
                                    transforms_he=None, 
                                    transforms_if=None)
_,t = dataset_train[0]
print("dataset_train size: ", len(dataset_train), "HE-IF pairs", t.dtype, t.min(), t.max())


#### Calculating percentiles batch

In [ ]:
MIN_PERCENTILE = 0.17
MAX_PERCENTILE = 99.99

Testing calculation with 1 sample

In [ ]:
# Before log10 
# Defininf axis with np.percentile: axis (1,0) returns 19 list element for first axis: x=1, y =0 ?
t_min = np.array([np.percentile(dataset_train[idx][1], MIN_PERCENTILE, axis=(1,0)) 
                       for idx in range(1)])

t_max = np.array([np.percentile(dataset_train[idx][1], MAX_PERCENTILE, axis=(1,0)) 
                       for idx in range(1)])
print("Before log10".center(60,"-"))
print(t_min.shape, t_min.min(), t_max.max()) 

# After log10 
t_min = np.mean([np.percentile(
                       np.log10(dataset_train[idx][1], where=(dataset_train[idx][1] != 0)), MIN_PERCENTILE, axis=(1,0)) 
                       for idx in range(1)], axis = 0)

t_max = np.mean([np.percentile(
                       np.log10(dataset_train[idx][1], where=(dataset_train[idx][1] != 0)), MAX_PERCENTILE, axis=(1,0)) 
                       for idx in range(1)], axis = 0)
print("After log10".center(60,"-"))
print(t_min.shape, t_min.min(), t_max.max()) 

Calculating percentiles for the entire training data

In [ ]:
import time
import datetime


start_time = time.time()
# specify lower and upper percentile cutoffs
lower_cutoff_log = np.mean([np.percentile(
                       np.log10(dataset_train[idx][1], where=(dataset_train[idx][1] != 0)), MIN_PERCENTILE, axis=(1,0)).ravel() 
                       for idx in range(len(dataset_train))], axis = 0
       )
upper_cutoff_log = np.mean([np.percentile(
                       np.log10(dataset_train[idx][1], where=(dataset_train[idx][1] != 0)), MAX_PERCENTILE, axis=(1,0)).ravel()
                       for idx in range(len(dataset_train))], axis = 0
       )
total_time = time.time() - start_time
print("total processing time: ", total_time)                                  
assert lower_cutoff_log.shape == upper_cutoff_log.shape
print(lower_cutoff_log, upper_cutoff_log)

Since we do not have the WSI but only the patches, I compute the mean of above all patches using comprenhension lists. 
Then apply log10 and verify the array values, which are rescaled back to ~0-4

In [ ]:
img_original = np.mean([dataset_train[idx][1] for idx in range(len(dataset_train))], axis = 0)
print("Mean IF img_original over all dataloader: ", img_original.shape, "Min value: ",  img_original.min(),"Max Value: ", img_original.max())
log_img_original = np.log10(img_original, where=(img_original != 0))
print("Mean IF log_img_original over all dataloader: ", log_img_original.shape, "Min value: ",  log_img_original.min(),"Max Value: ", log_img_original.max())

In [ ]:
len([dataset_train[idx][1] for idx in range(len(dataset_train))])

In [ ]:
marker = ['Hoechst','AF1','CD31','CD45','CD68','Blank','CD4',
          'FOXP3','CD8a','CD45RO','CD20','PD-L1','CD3e','CD163',
          'E-Cadherin','PD-1','Ki-67','Pan-CK','SMA'
         ]
# format plot grid
numRows = 4
numColumns = 6
grid_dims = (numRows, numColumns)
# initialize figure canvas
fig_orig = plt.figure(figsize=(12, 8.5))
fig_log = plt.figure(figsize=(12, 8.5))
fig_clip = plt.figure(figsize=(12, 8.5))

cutoffs = {}
for e, marker_name in enumerate(marker):
    # add channel cutoffs to dict
    cutoffs[marker_name]=(lower_cutoff_log[e], upper_cutoff_log[e])
    # scale 0.17th and 99.99th percentile between 0 and 1
    # Note: this will cause outlier pixels below the 0.17th percentile and above
    # the 99.99th to take values <0 and >1, respectively
    img = img_original[:,:,e]
    log_img = log_img_original[:,:,e]
    rescaled_log_img = (
        (((1-0)*(log_img.ravel()-lower_cutoff_log[e])) /
         (upper_cutoff_log[e]-lower_cutoff_log[e])
         ) + 0).reshape(log_img.shape)
    
    # clip outliers to lower and upper percentile cutoffs (i.e., 0-1)
    clip_rescaled_log_img = np.clip(a=rescaled_log_img, a_min=0, a_max=1)
    
    # add channel subplot to figures
    ax_orig = fig_orig.add_subplot(grid_dims[0], grid_dims[1], e + 1)
    ax_log = fig_log.add_subplot(grid_dims[0], grid_dims[1], e + 1)
    ax_clip = fig_clip.add_subplot(grid_dims[0], grid_dims[1], e + 1)
    fig_orig.tight_layout() #**ADDING**: vertical padding
    fig_log.tight_layout() #**ADDING**: vertical padding

    # plot original channel histogram
    vals, bins, patches = ax_orig.hist(
        img.ravel(), bins=60, color='tab:blue', alpha=0.7, rwidth=0.85
        )
    ax_orig.title.set_text(marker_name)
    
    # plot log-transformed channel histogram
    vals, bins, patches = ax_log.hist(
        log_img.ravel(), bins=60, color='tab:blue', alpha=0.7, rwidth=0.85
        )
    ax_log.vlines(
        x=[np.percentile(log_img.ravel(), 0.17),
           np.percentile(log_img.ravel(), 99.99)],
        ymin=0, ymax=vals.max(), color='tab:red'
           )
    ax_log.title.set_text(marker_name)
    # plot normalized channel histogram
    vals, bins, patches = ax_clip.hist(
        clip_rescaled_log_img.ravel(), bins=60,
        color='tab:blue', alpha=0.7, rwidth=0.85
        )
    ax_clip.title.set_text(marker_name)#**ADDING**: vertical padding
    
plt.xticks(fontsize=5)
plt.yticks(fontsize=5)
plt.subplots_adjust(bottom=0.9, top=0.99, left=0.01, right=0.99, hspace=10, )
plt.tight_layout()
plt.show()
fig_orig.savefig(os.path.join(save_dir, 'log_hists_orig.pdf'))
fig_log.savefig(os.path.join(save_dir, 'log_hists_log.pdf'))
fig_clip.savefig(os.path.join(save_dir, 'log_hists_clip.pdf'))
plt.close('all')

# save cutoffs to disk
with open(os.path.join(save_dir, 'cutoffs.pkl'), 'wb') as handle:
    pickle.dump(cutoffs, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Calculating minimum based on Percentile and the WSI

https://github.com/Yu-AnChen/orion-ml-meeting-notes/blob/main/20220916/file_vars.py

In [5]:
MIN_PERCENTILE = 0.17
MAX_PERCENTILE = 99.9#99.99

# CRC01
full_img = "/n/pfister_lab2/Lab/vcg_biology/ORION/ORION-IF-1/P37_S34_A24_C59kX_E15@20220107_202112_212579.ome.tiff"
root = "/net/coxfs01/srv/export/coxfs01/pfister_lab2/share_root/Lab/scajas/DATASETS/DATASET_pix2pix/train"
!ls /n/pfister_lab2/Lab/vcg_biology/ORION/ORION-IF-1

Celltable_C1.csv
Celltable_C1-YC.csv
Celltable_C2.csv
Celltable_C3.csv
Celltable_C40B1.csv
Celltable_C40B2.csv
Celltable_C4.csv
Celltype_OrionCRC_20220211.xlsx
P37_S24_Full_A24_C59kX_E15@20220104_223653_988272.ome.tiff
P37_S25_Full_A24_C59kX_E15@20220105_001442_622814.ome.tiff
P37_S25_Scan_20220114_131907_01x3x00400.rcpnl
P37_S26_Lung_A24_C59kX_E15@20220105_224440_730242.ome.tiff
P37_S26_NSCLC_A24_C59kX_E15@20220105_001550_901031.ome.tiff
P37_S27_Full_A24_C59kX_E15@20220106_002943_210932.ome.tiff
P37_S28_A24_C59kX_E15@20220106_014116_878434.ome.tiff
P37_S29_A24_C59kX_E15@20220106_014304_946511.ome.tiff
P37_S30_A24_C59kX_E15@20220106_014319_409148.ome.tiff
P37_S31_A24_C59kX_E15@20220106_014409_014236.ome.tiff
P37_S32_A24_C59kX_E15@20220106_014630_553652.ome.tiff
P37_S33_A24_C59kX_E15@20220107_180446_881530.ome.tiff
P37_S34_A24_C59kX_E15@20220107_202112_212579.ome.tiff
P37_S35_A24_C59kX_E15@20220108_012037_490594.ome.tiff
P37_S36_A24_C59kX_E15@20220108_012058_082564.ome.tiff
P37_S37_A24_

In [ ]:
from tifffile import imread

marker = ['Hoechst','AF1','CD31','CD45','CD68','Blank','CD4',
          'FOXP3','CD8a','CD45RO','CD20','PD-L1','CD3e','CD163',
          'E-Cadherin','PD-1','Ki-67','Pan-CK','SMA'
         ]

# save directory
save_dir = "/net/coxfs01/srv/export/coxfs01/pfister_lab2/share_root/Lab/scajas/pytorch-CycleGAN-and-pix2pix_master/CRC01_distributions_title_fixed"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# format plot grid
numRows = 4
numColumns = 6
grid_dims = (numRows, numColumns)
# initialize figure canvas
fig_orig = plt.figure(figsize=(12, 8.5))
fig_log = plt.figure(figsize=(12, 8.5))
fig_clip = plt.figure(figsize=(12, 8.5))

cutoffs = {}
for e, marker_name in enumerate(marker):
    # add channel cutoffs to dict
    #cutoffs[marker_name]=(lower_cutoff_log[e], upper_cutoff_log[e])
    # scale 0.17th and 99.99th percentile between 0 and 1
    # Note: this will cause outlier pixels below the 0.17th percentile and above
    # the 99.99th to take values <0 and >1, respectively
    # Read channel
    img = imread(full_img, key=e)
    #img =img_original[:,:,e] 
    print("Mean IF img_original over all dataloader: ", img.shape, "Min value: ",  img.min(),"Max Value: ", img.max())
    # log-transform image
    log_img = np.log10(img, where=(img != 0))
    print("Mean IF log_img over all dataloader: ", log_img.shape, "Min value: ",  log_img.min(),"Max Value: ", log_img.max())
    # specify lower and upper percentile cutoffs
    lower_cutoff_log = np.percentile(log_img.ravel(), 0.17)
    upper_cutoff_log = np.percentile(log_img.ravel(), 99.99)
    
    # add channel cutoffs to dict
    cutoffs[marker_name] = (lower_cutoff_log, upper_cutoff_log)
    
    # scale 0.17th and 99.99th percentile between 0 and 1
    # Note: this will cause outlier pixels below the 0.17th percentile and above
    # the 99.99th to take values <0 and >1, respectively
    rescaled_log_img = (
        (((1-0)*(log_img.ravel()-lower_cutoff_log)) /
         (upper_cutoff_log-lower_cutoff_log)
         ) + 0).reshape(log_img.shape)

    #rescaled_log_img = (
    #    (((1-0)*(log_img.ravel()-lower_cutoff_log[e])) /
    #     (upper_cutoff_log[e]-lower_cutoff_log[e])
    #     ) + 0).reshape(log_img.shape)
    
    # clip outliers to lower and upper percentile cutoffs (i.e., 0-1)
    clip_rescaled_log_img = np.clip(a=rescaled_log_img, a_min=0, a_max=1)
    
    # add channel subplot to figures
    ax_orig = fig_orig.add_subplot(grid_dims[0], grid_dims[1], e + 1)
    ax_log = fig_log.add_subplot(grid_dims[0], grid_dims[1], e + 1)
    ax_clip = fig_clip.add_subplot(grid_dims[0], grid_dims[1], e + 1)
    fig_orig.tight_layout() #**ADDING**: vertical padding
    fig_log.tight_layout() #**ADDING**: vertical padding

    # plot original channel histogram
    vals, bins, patches = ax_orig.hist(
        img.ravel(), bins=60, color='tab:blue', alpha=0.7, rwidth=0.85
        )
    ax_orig.title.set_text(marker_name)

    # plot log-transformed channel histogram
    vals, bins, patches = ax_log.hist(
        log_img.ravel(), bins=60, color='tab:blue', alpha=0.7, rwidth=0.85
        )
    ax_log.vlines(
        x=[np.percentile(log_img.ravel(), 0.17),
           np.percentile(log_img.ravel(), 99.99)],
        ymin=0, ymax=vals.max(), color='tab:red'
           )
    ax_log.title.set_text(marker_name)

    # plot normalized channel histogram
    vals, bins, patches = ax_clip.hist(
        clip_rescaled_log_img.ravel(), bins=60,
        color='tab:blue', alpha=0.7, rwidth=0.85
        )
    ax_clip.title.set_text(marker_name)

plt.xticks(fontsize=7)
plt.yticks(fontsize=7)
plt.subplots_adjust(bottom=0.01, top=0.99, left=0.01, right=0.99, hspace=0.2)
plt.tight_layout()
fig_orig.savefig(os.path.join(save_dir, 'log_hists_orig.pdf'))
fig_log.savefig(os.path.join(save_dir, 'log_hists_log.pdf'))
fig_clip.savefig(os.path.join(save_dir, 'log_hists_clip.pdf'))
plt.close('all')

# save cutoffs to disk
with open(os.path.join(save_dir, 'cutoffs.pkl'), 'wb') as handle:
    pickle.dump(cutoffs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

Mean IF img_original over all dataloader:  (53770, 64422) Min value:  0 Max Value:  62173
Mean IF log_img over all dataloader:  (53770, 64422) Min value:  0.0 Max Value:  4.793602


In [ ]:
!ls /n/pfister_lab2/Lab/vcg_biology/ORION/ORION-PATCH/C1-C40-patches
!ls /n/pfister_lab2/Lab/vcg_biology/ORION/
#!ls /n/pfister_lab2/Lab/vcg_biology/ORION/ORION-PATCH/C1-C40-patches
P37_S34_A24_C59kX_E15

In [ ]:
!ls /n/pfister_lab2/Lab/vcg_biology/ORION/ORION-IF-4